In [1]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [259]:
print("Data read started...")
data = pd.read_csv("result9.csv")
data = data.as_matrix()
print ("Data read finished.")

print(data.shape)

touch_only = 'false'
touch_heart = 'true'

Data read started...
Data read finished.
(313, 15)


In [260]:
data = np.delete(data, 13, 1)    # TouchNumber
data = np.delete(data, 12, 1)    # T/S
data = np.delete(data, 11, 1)    # TouchTimestamp
data = np.delete(data, 4, 1)     # RRInterval
data = np.delete(data, 3, 1)     # hpvPower
data = np.delete(data, 2, 1)     # Heartrate
data = np.delete(data, 1, 1)     # Timestamp
data.shape

if(touch_only == 'true'):
    data = np.delete(data, 7, 1)     # SDNN
    data = np.delete(data, 3, 1)     # Burg
    data = np.delete(data, 2, 1)     # ReactionTime

if(touch_heart == 'true'):
    data = np.delete(data, 7, 1)     # ReactionTime
    
data.shape

(313, 7)

# Dictionary for the levels

In [261]:
level=["NbackSym1","NbackSym2","NbackSym3"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [262]:
for i in range (1,data.shape[1]):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [263]:
for i in range(len(data)):
    data[i, 0] = level2int[data[i, 0]]

x_data = data[:, 1:]
y_data = data[:, 0]

# One-hot encoding

In [264]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y_one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()

# Shuffle data

In [265]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [266]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [267]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(219, 6) (219, 3) (46, 6) (46, 3) (48, 6) (48, 3)


# Build the net

In [268]:
model = Sequential()

model.add(Dense(200, input_shape=(x_train.shape[1],)))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation='softmax'))

In [269]:
batch_size = 50
epochs = 200
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 219 samples, validate on 46 samples
Epoch 1/200
219/219 [==============================] - 0s - loss: 1.0216 - acc: 0.5297 - val_loss: 0.9096 - val_acc: 0.7174
Epoch 2/200
219/219 [==============================] - 0s - loss: 0.8077 - acc: 0.7260 - val_loss: 0.6833 - val_acc: 0.7391
Epoch 3/200
219/219 [==============================] - 0s - loss: 0.6100 - acc: 0.7443 - val_loss: 0.4975 - val_acc: 0.8043
Epoch 4/200
219/219 [==============================] - 0s - loss: 0.4929 - acc: 0.7991 - val_loss: 0.3593 - val_acc: 0.8478
Epoch 5/200
219/219 [==============================] - 0s - loss: 0.4261 - acc: 0.8174 - val_loss: 0.3126 - val_acc: 0.8913
Epoch 6/200
219/219 [==============================] - 0s - loss: 0.3592 - acc: 0.8676 - val_loss: 0.2925 - val_acc: 0.8696
Epoch 7/200
219/219 [==============================] - 0s - loss: 0.3851 - acc: 0.8584 - val_loss: 0.2780 - val_acc: 0.8696
Epoch 8/200
219/219 [==============================] - 0s - loss: 0.3214 - acc: 0.8539 

In [270]:
model.evaluate(x_test, y_test, batch_size=batch_size)

48/48 [==============================] - 0s


[0.00044534937478601933, 1.0]

In [271]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 1(1back)', 'class 2(2back)', 'class3(3back)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

32/48 [===================>..........] - ETA: 0s[2 2 1 1 1 2 2 0 0 1 0 2 2 2 2 0 0 0 1 1 1 2 2 2 2 0 2 2 1 2 2 2 1 2 1 2 0
 1 1 0 1 0 1 0 2 2 0 1]
32/48 [===================>..........] - ETA: 0s                precision    recall  f1-score   support

class 1(1back)       1.00      1.00      1.00        12
class 2(2back)       1.00      1.00      1.00        15
 class3(3back)       1.00      1.00      1.00        21

   avg / total       1.00      1.00      1.00        48

[[12  0  0]
 [ 0 15  0]
 [ 0  0 21]]
